In [5]:
import pandas as pd
import geopandas as gpd

import warnings
warnings.filterwarnings('ignore')

import os

## Cargar, corregir y unificar archivos

En esta notebook se genera y guarda el archivo `./mapaelectoral/circuitos-CNE-TTGL.geojson`


In [8]:
## Usar circuitos electorales de Tartagalensis

## Descargar los archivos de circuitos de git.
! git clone --no-checkout --filter=tree:0 https://github.com/tartagalensis/circuitos_electorales_AR
os.chdir('./circuitos_electorales_AR')
! git sparse-checkout set geojson
os.chdir('./../')


fatal: destination path 'circuitos_electorales_AR' already exists and is not an empty directory.


In [72]:

from pathlib import Path

# path_circuitos_geo = './mapaelectoral/geojson_CNE/'
path_circuitos_geo = './circuitos_electorales_AR/geojson/'
p = Path(path_circuitos_geo)

result_list = []

for file in sorted(p.glob('**/*')):
    print(file.name)
    gdf = gpd.read_file(path_circuitos_geo + file.name)
    
    ## Estas dos provincias tendrian algun problemita en el archivo de TTgalensis
    if file.name in ['SAN_JUAN.geojson', 'MISIONES.geojson']:
        gdf_ = gdf.copy()
        gdf_['circuito'] = gdf_['circuito'].astype(str).str.zfill(5)
        gdf_['codprov'] = gdf_['distrito'].astype(str).str.zfill(2)
        gdf_['coddepto'] = gdf_['indec_d'].astype(str).str.zfill(3)

        gdf = gdf_[['circuito', 'codprov', 'coddepto', 'geometry']]
        
    if file.name == 'CORDOBA.geojson':
        gdf['codprov'] = '04'

    ## Unify column names
    gdf = gdf.rename(columns = {'coddistrito': 'codprov', 'codseccion': 'coddepto'})
    
#     if gdf.shape != (0, 1): ## Si el archivo esta OK
#         print(gdf.provincia.unique()[0])

#         ## Correccion de errores
#         gdf.loc[gdf.departamen == 'Colon', 'indec_d'] = '008'
#         gdf.loc[gdf.departamen == 'Concordia', 'indec_d'] = '015'
#         gdf.loc[gdf.departamen == 'Villaguay', 'indec_d'] = '113'
#         gdf.loc[gdf.departamen == 'Pehuenches', 'indec_d'] = '091'
#         gdf.loc[gdf.departamen == 'Cañuelas', 'indec_d'] = '134'
#         gdf.loc[gdf.departamen == 'Esteban Echeverria', 'indec_d'] = '260'
#         gdf.loc[gdf.departamen == 'Florencio Varela', 'indec_d'] = '274'
#         gdf.loc[gdf.departamen == 'Magdalena', 'indec_d'] = '505'
#         gdf.loc[gdf.departamen == 'Morón', 'indec_d'] = '568'
#         gdf.loc[gdf.departamen == 'Pilar', 'indec_d'] = '638'
#         gdf.loc[gdf.departamen == 'San Vicente', 'indec_d'] = '778'
#         gdf.loc[gdf.departamen == '12 de Octubre', 'indec_d'] = '036'
#         gdf.loc[(gdf.provincia == 'Chaco')
#                     & (gdf.departamen == 'Almirante Brown'), 'indec_d'] = '043'

    result_list += [gdf]


circuitos_ARG = pd.concat(result_list)

CABA.geojson
CATAMARCA.geojson
CHACO.geojson
CHUBUT.geojson
CORDOBA.geojson
CORRIENTES.geojson
ENTRERIOS.geojson
FORMOSA.geojson
JUJUY.geojson
LAPAMPA.geojson
LA_RIOJA.geojson
MENDOZA.geojson
MISIONES.geojson
NEUQUEN.geojson
PBA.geojson
RIO_NEGRO.geojson
SALTA.geojson
SANLUIS.geojson
SANTACRUZ.geojson
SANTAFE.geojson
SANTIAGO_DEL_ESTERO.geojson
SAN_JUAN.geojson
TIERRA_DEL_FUEGO.geojson
TUCUMAN.geojson


### Arreglar geometrias con problema

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from shapely.validation import make_valid

def fix_geom(in_feature):

    # avoid changing original geodf
    in_feature = in_feature.copy(deep=True)    
        
    # drop any missing geometries
    in_feature = in_feature[~(in_feature.is_empty)]
    
    # Repair broken geometries
    for index, row in in_feature.iterrows(): # Looping over all polygons
        if row['geometry'].is_valid:
            next
        else:
            fix = make_valid(row['geometry'])

            try:
                in_feature.loc[[index],'geometry'] =  fix # issue with Poly > Multipolygon
            except ValueError:
                in_feature.loc[[index],'geometry'] =  in_feature.loc[[index], 'geometry'].buffer(0)
    return in_feature

circuitos_ARG = fix_geom(circuitos_ARG.dropna(subset = 'geometry'))

In [ ]:
circuitos_ARG.to_file('./mapaelectoral/circuitos-CNE-TTGL.geojson', driver='GeoJSON')


## Ubicar Circuitos en Dptos y Provs de IGN a traves de Spatial Join

In [9]:
## Si este paso falla, se pueden bajar manualmente los archivos y abrirlos localmente.
import geopandas as gpd

try: 
    dptos_url = u'https://dnsg.ign.gob.ar/apps/api/v1/capas-sig/Geodesia+y+demarcación/Límites/departamento/json'
    dptos_IGN = gpd.read_file(dptos_url)[['gid', 'in1', 'nam', 'geometry']]

    # Agregar código y nombres de provincia
    provs_url = 'https://dnsg.ign.gob.ar/apps/api/v1/capas-sig/Geodesia+y+demarcación/Límites/provincia/json'
    provs_IGN = gpd.read_file(provs_url)[['gid', 'in1', 'nam', 'geometry']]

except:
    dptos_IGN = gpd.read_file('./datos/departamento.json')[['gid', 'in1', 'nam', 'geometry']]
    provs_IGN = gpd.read_file('./datos/provincia.json')[['gid', 'in1', 'nam', 'geometry']]


## Renombrar para evitar los nombres de columna repetido entre Provs y Dptos
# Create a dictionary of old column names and new column names
dpto_colname_mapping = {col: col + '_dpto' for col in [col for col in dptos_IGN.columns if col != 'geometry']}
prov_colname_mapping = {col: col + '_prov' for col in [col for col in provs_IGN.columns if col != 'geometry']}

# Rename the columns using the dictionary
dptos_IGN = dptos_IGN.rename(columns=dpto_colname_mapping)
provs_IGN = provs_IGN.rename(columns=prov_colname_mapping)

In [50]:
circuitos_ARG['coddepto'] = circuitos_ARG['coddepto'].fillna('000')
circuitos_ARG['codprov'] = circuitos_ARG['codprov'].fillna('00')

In [53]:
import geopandas as gpd

# Perform spatial intersection between circuitos_ARG and provs_IGN
intersections = gpd.overlay(circuitos_ARG, dptos_IGN, how='intersection')
intersections['area'] = intersections.area # Compute the area of each intersection

largest_intersections = intersections.groupby(['codprov', 'coddepto', 'circuito']).apply(lambda x: x.nlargest(1, 'area')).reset_index(drop=True)
circuitos_dpto = largest_intersections[['codprov', 'coddepto', 'circuito', 'in1_dpto', 'nam_dpto', 'geometry']]

In [58]:
intersections = gpd.overlay(circuitos_dpto, provs_IGN, how='intersection')
intersections['area'] = intersections.area # Compute the area of each intersection

largest_intersections = intersections.groupby(['codprov', 'coddepto', 'circuito']).apply(lambda x: x.nlargest(1, 'area')).reset_index(drop=True)
circuitos_dpto_prov = largest_intersections[['in1_prov', 'nam_prov', 'in1_dpto', 'nam_dpto', 'circuito', 'codprov', 'coddepto', 'geometry']]

In [83]:
circuitos_ARG.nunique()

circuito    2047
codprov       25
coddepto     138
seccion        8
geometry    5550
dtype: int64

In [85]:
circuitos_dpto_prov.nunique()

in1_prov      24
nam_prov      24
in1_dpto     527
nam_dpto     445
circuito    2045
codprov       25
coddepto     138
geometry    5526
dtype: int64

In [86]:
circuitos_dpto_prov.to_file('./datos/circuitos_dpto_prov.geojson', driver='GeoJSON')